In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys, os, random, glob, cv2, math, urllib
from PIL import Image
from scipy import stats

import fastai
from fastai.vision import *
from fastai.imports import *
from fastai.vision.data import *
from fastai.vision.all import *


package_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

np.random.seed(42) 

In [2]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
        
!cp '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth' '/root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth'
!cp '../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth' '/root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth'
!cp '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth' '/root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth'

In [3]:
train = pd.read_csv("../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train.csv")
train['image_id']=train['image_id'].map(lambda x : '../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/'+x+'.jpg')
train.head()

,image_id,breed
0,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/a8b3ad1dde.jpg,nerodia-erythrogaster
1,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/8b492b973d.jpg,pantherophis-vulpinus
2,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/929b99ea92.jpg,thamnophis-sirtalis
3,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/bbac7385e2.jpg,pantherophis-obsoletus
4,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/ef776b1488.jpg,agkistrodon-contortrix


In [4]:
path = Path('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset')


# Albumentations

In [5]:
import albumentations as A
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [6]:
p1=0.5
def get_train_aug(): return A.Compose([
        A.RandomCrop(224,224, p=0),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Transpose(p=0.5),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.3),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.Blur(blur_limit=3,p=0.3),
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(
            hue_shift_limit=0.2, 
            sat_shift_limit=0.2, 
            val_shift_limit=0.2, 
            p=0.5),
        A.CoarseDropout(p=0.5),
        A.Cutout(p=0.5)
    ],p=1)

def get_valid_aug(): return A.Compose([
    A.CenterCrop(224,224, p=1.),
    A.Resize(224,224)
], p=0.)

# Data Preparation

In [7]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)

train['fold'] = -1
for i, (_, test_index) in enumerate(folds.split(train.image_id.values, train.breed.values)):
    train.iloc[test_index, -1] = i
train.head()

,image_id,breed,fold
0,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/a8b3ad1dde.jpg,nerodia-erythrogaster,5
1,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/8b492b973d.jpg,pantherophis-vulpinus,1
2,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/929b99ea92.jpg,thamnophis-sirtalis,9
3,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/bbac7385e2.jpg,pantherophis-obsoletus,7
4,../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train/ef776b1488.jpg,agkistrodon-contortrix,8


In [8]:
item_tfms = [Resize(300), AlbumentationsTransform(get_train_aug(), get_valid_aug())]
batch_tfms=[Normalize.from_stats(*imagenet_stats)]
mixup = MixUp(0.3)

In [9]:
def get_data(fold=0, size=224,bs=32):
    return DataBlock(blocks=(ImageBlock,CategoryBlock),
                     getters=[ColReader('image_id'),ColReader('breed')],n_inp=1,
                       splitter=IndexSplitter(train[train.fold == fold].index),
                       item_tfms=[Resize(size), AlbumentationsTransform(get_train_aug(), get_valid_aug())],
                       batch_tfms=batch_tfms,
                      ).dataloaders(train, bs=bs)

# Training

In [10]:
all_preds = []
test_files = get_image_files(path/'test')
import gc
for i in range(10):
    print("Fold {}".format(i))
    model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=35)
    data = get_data(i,300,64)
    learn = Learner(data, model,metrics=accuracy).to_fp16()
    learn.path = Path('/kaggle/working/')
    learn.fine_tune(5, cbs=[ EarlyStoppingCallback(monitor='valid_loss',patience=2),SaveModelCallback(monitor='accuracy'),mixup])
    
    #data = get_data(i,300,32)
    #learn.fine_tune(5, cbs=[ EarlyStoppingCallback(monitor='valid_loss',patience=2),SaveModelCallback(monitor='accuracy')])
    
    learn.dls=get_data(i,512,24)
    learn.dls.bs=24
    learn.fine_tune(5, cbs=[ EarlyStoppingCallback(monitor='valid_loss',patience=2),SaveModelCallback(monitor='accuracy'),mixup])
    
    tst_dl = learn.dls.test_dl(test_files)
    preds, _ = learn.tta(dl=tst_dl)
    all_preds.append(learn.dls.vocab[np.argmax(preds,1)])
    learn.save("fold_"+str(i))
    del learn
    torch.cuda.empty_cache()
    gc.collect()
    
m = stats.mode(all_preds)

Fold 0
Loaded pretrained weights for efficientnet-b4


epoch,train_loss,valid_loss,accuracy,time
0,3.164474,3.409638,0.183303,01:46


Better model found at epoch 0 with accuracy value: 0.1833030879497528.


epoch,train_loss,valid_loss,accuracy,time
0,2.575049,2.568687,0.297641,01:39
1,2.472322,2.360275,0.352087,01:39
2,2.279999,2.088909,0.424682,01:39
3,1.991768,1.827738,0.480944,01:40
4,1.756843,1.745092,0.500907,01:39


Better model found at epoch 0 with accuracy value: 0.2976406514644623.
Better model found at epoch 1 with accuracy value: 0.3520871102809906.
Better model found at epoch 2 with accuracy value: 0.4246824085712433.
Better model found at epoch 3 with accuracy value: 0.4809437394142151.
Better model found at epoch 4 with accuracy value: 0.5009074211120605.


...

Better model found at epoch 0 with accuracy value: 0.36181819438934326.
Better model found at epoch 1 with accuracy value: 0.3909091055393219.
Better model found at epoch 2 with accuracy value: 0.4490908980369568.
Better model found at epoch 3 with accuracy value: 0.4872727394104004.
Better model found at epoch 4 with accuracy value: 0.5018181800842285.


epoch,train_loss,valid_loss,accuracy,time
0,2.631206,3.229875,0.145455,04:23


Better model found at epoch 0 with accuracy value: 0.145454540848732.


epoch,train_loss,valid_loss,accuracy,time
0,2.248964,2.053105,0.452727,04:21
1,2.343644,2.321923,0.358182,04:21
2,2.087128,2.072469,0.421818,04:22


Better model found at epoch 0 with accuracy value: 0.4527272582054138.
No improvement since epoch 0: early stopping


In [11]:
image_id=list(map(lambda x: x.name, test_files))
test = pd.read_csv('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test.csv')
submission= test.copy()
test['image_id']=image_id
test['label']=m[0][0]

In [12]:
submission['breed'] = submission['image_id']
submission['breed'] = submission['breed']. map(lambda x: test[test['image_id']==x+'.jpg'].values[0][1])
submission.to_csv('submission.csv',index=False)
submission.head()

,image_id,breed
0,7ede553357,pantherophis-obsoletus
1,3500b219e4,agkistrodon-piscivorus
2,d43a78d6d9,pantherophis-obsoletus
3,1f36f26994,storeria-dekayi
4,8cb85fc58e,pantherophis-obsoletus
